In [2]:
import pickle
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, load_model
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
import pandas as pd
import numpy as np

In [3]:
with open('dataset_emotion_preprocessed.pickle', 'rb') as f:
   dataset_emotion = pickle.load(f)

In [4]:
dataset_emotion.columns

Index(['tokenized_tweets', 'emotion', 'processed'], dtype='object')

In [6]:
dataset_emotion["emotion"].unique()

array(['sadness', 'happiness', 'neutral', 'worry', 'surprise', 'love'],
      dtype=object)

In [7]:
tokenizer = Tokenizer(num_words=1500, split=' ')

tokenizer.fit_on_texts(dataset_emotion['processed'].values)

X = tokenizer.texts_to_sequences(dataset_emotion['processed'])

X = pad_sequences(X)

In [8]:
files = open("tokenizer_emotion.pickle", "wb")
pickle.dump(tokenizer, files)

In [9]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(2000, 300, input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(6,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 30, 300)           600000    
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 30, 300)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 196)               389648    
_________________________________________________________________
dense (Dense)                (None, 6)                 1182      
Total params: 990,830
Trainable params: 990,830
Non-trainable params: 0
_________________________________________________________________
None


In [10]:
Y = pd.get_dummies(dataset_emotion['emotion']).values

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.20, random_state=42)

In [10]:
batch_size = 32
model.fit(X_train, y_train, epochs = 50, batch_size=batch_size, verbose = 1)

Epoch 1/3
1495/1495 [==============================] - 190s 120ms/step - loss: 1.3228 - accuracy: 0.4548
Epoch 2/3
1495/1495 [==============================] - 172s 115ms/step - loss: 1.1481 - accuracy: 0.5315
Epoch 3/3
1495/1495 [==============================] - 182s 122ms/step - loss: 1.1157 - accuracy: 0.5443


In [11]:
model.save('emotions_model_lstm_temp')

INFO:tensorflow:Assets written to: emotions_model_lstm_temp\assets


In [19]:
score, accuracy = model.evaluate(X_test, y_test, verbose=1)

374/374 [==============================] - 9s 24ms/step - loss: 1.1606 - accuracy: 0.5256
